In [111]:
import pandas as pd
import numpy as np
import pandasgui as pg 

curated_vars = pd.read_csv("../data/row_curateddata.csv", dtype={"Pubmed_id":"object"})
snp_table = pd.read_csv("../data/july20th_previous_database/snps_July20.csv")
pmid_table = pd.read_csv("../data/july20th_previous_database/study_July20.csv", dtype={"reference_id":"object"})

In [112]:
filter_list = []
for var in curated_vars.Variant_ID:
    if "*" in var:
        filter_list.append(True)
    else:
        filter_list.append(False)

star_alleles  = curated_vars[filter_list]
snp = curated_vars[np.invert(filter_list)]
snp.drop(columns=["Unnamed: 0"], inplace=True)
snp.rename(columns={"Variant_ID":"rs_id"}, inplace=True)

In [135]:
# add genes 
genes = snp_table[["rs_id", "gene_id" ]]
new_snp = pd.merge(snp, genes, on="rs_id").drop_duplicates()

# add reference 
pmid= pmid_table[["ref_id", "reference_id" ]]
new_snp = pd.merge(new_snp, pmid, left_on="Pubmed_id",right_on="reference_id").drop_duplicates()

# add the other columns 
last_merg_table = snp_table[["rs_id", "id", "reference_id", "drug_id", "ID_diseases", "source", "id_in_source"]]

In [128]:
pg.show(new_snp)

In [148]:
raw_merged = pd.merge(new_snp, last_merg_table, how="left",
                      left_on=["rs_id", "ref_id"], right_on = ["rs_id", "reference_id"])
raw_merged = raw_merged[["id","rs_id", "gene_id", "drug_id", "ID_diseases", "phenotype","reference_id_y", "p-value",
           "source", "id_in_source", "region", "Ethnicity", "Notes", "origin_of_participants"  ]].drop_duplicates()

In [149]:
raw_merged.to_csv("../data/new_curated_snps.csv", index=False)

In [157]:
star = pd.read_csv('../data/july20th_previous_database/star_July20.csv')
#star_alleles.drop(columns=["Unnamed: 0"], inplace=True)
star_alleles.rename(columns={"Variant_ID":"star_annotation"}, inplace=True)

/home/houcem/env_module/modules/software/miniconda3/envs/jupyter_env/lib/python3.9/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [197]:
genes= star[["star_annotation", "gene_id" ]]
new_star = pd.merge(star_alleles, genes, on="star_annotation").drop_duplicates()

# add reference 
pmid= pmid_table[["ref_id", "reference_id" ]]
new_star = pd.merge(new_star, pmid, left_on="Pubmed_id",right_on="reference_id").drop_duplicates()

# add the other columns 
last_merg_table_star = star[["star_annotation", "id", "reference_id", "drug_id", "source", "id_in_source"]]

raw_merged_star = pd.merge(new_star, last_merg_table_star, how="left",
                      left_on=["star_annotation", "ref_id"], right_on = ["star_annotation", "reference_id"]).drop_duplicates()

raw_merged_star = raw_merged_star[["id", "star_annotation", "gene_id", "drug_id", "phenotype", "reference_id_y",
                "p-value", "source", "id_in_source", "region", "Ethnicity", "Notes", "origin_of_participants" ]]

In [198]:
raw_merged_star.to_csv("../data/new_curated_star.csv", index=False)

In [188]:
star.dtypes

id                         object
star_annotation            object
gene_id                    object
drug_id                    object
allele                     object
phenotype                  object
reference_id               object
P-value                    object
source                     object
id_in_source                int64
region                     object
Country_of_Participants    object
dtype: object

In [189]:
pg.show(raw_merged_star)

In [190]:
star

,id,star_annotation,gene_id,drug_id,allele,phenotype,reference_id,P-value,source,id_in_source,region,Country_of_Participants
0,STAR1,CYP1A2*1K,PHARGENE19,DRUG10,*1K,CYP1A2 *1K is associated with decreased metabo...,REF162,< 0.02,PharmGKB,827698390,Sub-Saharan African,Ethiopia
1,STAR2,CYP3A4*1,PHARGENE27,DRUG12,*1/*22 + *22/*22,CYP3A4 *1/*22 + *22/*22 is associated with con...,REF152,0.01,PharmGKB,1184746928,African American/Afro-Caribbean,USA
2,STAR3,CYP3A4*22,PHARGENE27,DRUG12,*1/*22 + *22/*22,CYP3A4 *1/*22 + *22/*22 is associated with con...,REF152,0.01,PharmGKB,1184746928,African American/Afro-Caribbean,USA
3,STAR196,CYP3A4*1,PHARGENE27,DRUG55,*1/*1B,CYP3A4 *1/*1B is associated with increased dos...,REF160,NaN,PharmGKB,1449170970,African American/Afro-Caribbean,USA
4,STAR197,CYP3A4*1B,PHARGENE27,DRUG55,*1/*1B,CYP3A4 *1/*1B is associated with increased dos...,REF160,NaN,PharmGKB,1449170970,African American/Afro-Caribbean,USA
...,...,...,...,...,...,...,...,...,...,...,...,...
198,STAR157,CYP2D6*36,PHARGENE26,DRUG36,*2/*36,CYP2D6 *2/*36 (assigned as extensive metaboliz...,REF76,NaN,PharmGKB,1446897151,African American/Afro-Caribbean,USA
199,STAR202,CYP2D6*2,PHARGENE26,DRUG57,*2/*41,CYP2D6 *2/*41 is associated with increased con...,REF59,NaN,PharmGKB,1449189944,Sub-Saharan African,Canada
200,STAR203,CYP2D6*41,PHARGENE26,DRUG57,*2/*41,CYP2D6 *2/*41 is associated with increased con...,REF59,NaN,PharmGKB,1449189944,Sub-Saharan African,Canada
201,STAR129,CYP2C19*1,PHARGENE23,DRUG40,*1/*17,CYP2C19 *1/*17 is associated with decreased co...,REF161,NaN,PharmGKB,1444828163,Sub-Saharan African,Cameroon
